In [0]:
  from pyspark.sql.functions import array, when, lit, current_timestamp
  
  class util:
    
    
    def handleErrors(df, df_errors, columns: list, function_name, error_action: str, error_value, replace_value=None):
      
      def writeErrorMessage(errorColumns):
        errorMessage = f"{function_name} failed on Column(s):"+", ".join([column_name for column_name in errorColumns if column_name is not None])
        return errorMessage
    
      udf_writeErrorMessage = udf(writeErrorMessage, StringType())
      
      # Generate filter
      filterStringErrors = ' or '.join([c_name+"__transformed__" +f"='{error_value}'" for c_name in columns])
      filterStringCleaned = ' and '.join([c_name+"__transformed__" +f"<>'{error_value}'" for c_name in columns])   
      
      
      # Get Error Dataframe
      temp_df_errors = df.where(filterStringErrors)
      temp_df_errors = temp_df_errors.withColumn("__DataProcessingError__", array(*[when(col(c_name+"__transformed__")==error_value, lit(c_name)) for c_name in columns]))\
                                    .withColumn("__DataProcessingError__", udf_writeErrorMessage(col("__DataProcessingError__")))
      temp_df_errors = temp_df_errors.withColumn("__Function__", lit(function_name))
      temp_df_errors = temp_df_errors.withColumn("__CleanseTimestamp__", current_timestamp())
      temp_df_errors = temp_df_errors.drop(*[c_name+"__transformed__" for c_name in columns])
      temp_df_errors = temp_df_errors.select(df_errors.columns)
      df_errors= df_errors.select(df_errors.columns)
      df_errors = df_errors.union(temp_df_errors)
      
      cleanedCols = []
      for sel_col in df.columns:
        if "__transformed__" not in sel_col: 
          cleanedCols.append(sel_col)
      
      # Get Cleaned DataFrame
      if error_action.lower() == "stop":
        if temp_df_errors is not None:
          raise Exception('Errors found and ErrorAction is Stop')
        else:
          df_cleaned = df.drop(*columns) # remove original columns 
          for c_name in columns:
            df_cleaned = df_cleaned.withColumnRenamed(c_name+"__transformed__", c_name)  # rename columns to original        
      
      elif error_action.lower() == "continue_and_drop_row":
        # Get Cleaned DataFrame
        df_cleaned = df.where(filterStringCleaned)
        df_cleaned = df_cleaned.drop(*columns) # remove original columns  
        for c_name in columns:
          df_cleaned = df_cleaned.withColumnRenamed(c_name+"__transformed__", c_name)    # rename columns to original  
      
      elif error_action.lower() == "continue_and_null_value":
        # Get Cleaned DataFrame
        df_cleaned = df.replace(error_value, value=None, subset=[c_name+"__transformed__" for c_name in columns])     
        df_cleaned = df_cleaned.drop(*columns) # remove original columns
        for c_name in columns:
          df_cleaned = df_cleaned.withColumnRenamed(c_name+"__transformed__", c_name)    # rename columns to original      
        
      elif error_action.lower() == "continue_and_replace_value":
        df_cleaned = df.replace(error_value, value=replace_value, subset=[c_name+"__transformed__" for c_name in columns])
        df_cleaned = df_cleaned.drop(*columns) # remove original columns
        for c_name in columns:
          df_cleaned = df_cleaned.withColumnRenamed(c_name+"__transformed__", c_name) # rename columns to original
      
      else:
        raise Exception(f'Invalid Error Action: {error_action}')
        
      df_cleaned = df_cleaned.select(cleanedCols) # ensure correct order 
      return df_cleaned, df_errors
  
    def safeUdf(fn, dtype, error_value):
      def _(*args):
          try:
            return fn(*args)
          except:
            # In future, return exception string
            return error_value
      return udf(_, dtype)